In [1]:
import numpy as np
import pandas as pd
import transformers
from transformers import AutoModel, BertTokenizerFast, BertConfig, BertModel
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.autograd
import torch.nn.functional as F
import torch
from torch import Tensor
from torch.nn.parameter import Parameter, UninitializedParameter
from torch.nn import functional as F

In [2]:
def func_collecting_tensors(step, tensor1, tensor2=None):
    '''собирает в тензор     '''
    if step == 0:
        return tensor1.unsqueeze(0)
    else:
        return torch.concatenate((tensor1, tensor2),0)
    


def Tucker_Decomposition(tensor):
    n1, n2, n3 = tensor.shape
    u1, _, _ = torch.svd(torch.reshape(tensor, (n1, -1)))
    u2, _, _ = torch.svd(torch.reshape(torch.permute(tensor, [1, 2, 0]), (n2, -1)))
    u3, _, _ = torch.svd(torch.reshape(torch.permute(tensor, [2, 0, 1]), (n3, -1)))
    return u1, u2, u3


def get_tucker_tensors(dict_layers):
    '''делает словарь где ключом будет слой, а значением будет тензор'''        
    dict_tensor = dict(zip(range(12), [[]]*12))
    for key in dict_layers.keys():
        dict_tensor[key].append(torch.cat(dict_layers[key], 2 ))
    return dict_tensor

In [3]:
class LinearFunctionSparseGrad(torch.autograd.Function):

        # Note that forward, setseup_context, and backward are @staticmethods
    @staticmethod
    def forward(ctx, input, weight, bias, grads, treshold, U, VT):
        treshold = treshold
        if ((len(grads) <30 )):

            ctx.save_for_backward(input, weight, bias)
            return  input @ weight.T + bias
        else:
            #u1, U, VT = Tucker_Decomposition(torch.cat(MyLayer.grads))
            ctx.save_for_backward(input, weight, bias, U, VT)
        ctx.size = input.shape[0]
        
        print (input.shape)
        print (weight.shape)
        print (bias.shape)
        print (len(grads)),
        print (treshold)
        print (U.shape, VT.shape)

        return  input @ U.T @ weight.T @ VT.T  + bias
    


    @staticmethod
    def backward(ctx, grad_output):

        if len(ctx.saved_tensors) == 3:
            input,weight, bias = ctx.saved_tensors
            grad_input = grad_weight = grad_bias = None
            if ctx.needs_input_grad[0]:
                grad_input = grad_output @ weight
            if ctx.needs_input_grad[1]:
                grad_weight =  grad_output.T @input
  
            if bias is not None and ctx.needs_input_grad[2]:
                grad_bias = grad_output
            
        elif len(ctx.saved_tensors) == 5:

            input,weight, bias, U, VT = ctx.saved_tensors
            grad_input = grad_weight = grad_bias = None
            if ctx.needs_input_grad[0]:
                grad_input = grad_output @ weight
            if ctx.needs_input_grad[1]:
                grad_weight = grad_output.T @input 
                grad_weight = U @  grad_weight @ VT
                grad_weight = torch.where(torch.abs(grad_weight) >= treshold, grad_weight, 0)  ## возвращаем градиент в каком пространстве?? VERY IMPORTANT
            if bias is not None and ctx.needs_input_grad[2]:
                grad_bias = grad_output
            
        return grad_input, grad_weight, grad_bias, None, None, None, None
        
        


In [23]:
class SparseGradLinear(torch.nn.Module):
    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.empty((out_features, in_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.grads = []
        self.treshold = 1e-3
        self.U = None
        self.VT = None
        
        
    def create_UV(self):
        if (len(self.grads)>30):
            u1, U, VT = Tucker_Decomposition(torch.cat(self.grads))
            self.U = U
            self.VT = VT
            self.weight = U@self.weight@VT
        else:
            print ("please do 30 optimizer steps")
            
        
        
    def from_linear(self, linear: nn.Linear, transpose=False):
        if transpose:
            self.weight = torch.nn.Parameter(linear.weight.data.T)
        else:
            self.weight = torch.nn.Parameter(linear.weight.data)
        self.bias = torch.nn.Parameter(linear.bias.data.clone()) if linear.bias is not None else None
        
        
        
    def forward(self, x):
        return LinearFunctionSparseGrad.apply(x, self.weight, self.bias, self.grads, self.treshold, self.U, self.VT)
            

In [24]:
x= torch.ones((1,10))
a = torch.nn.Parameter(torch.randn((10,10)))
b = torch.nn.Parameter(torch.randn((10,10)))
c = torch.nn.Parameter(torch.randn((1, 10)))
u = torch.randn((10,10)) 
vt = torch.randn((10,10))
treshold = 1e-3

In [25]:
MyLayer = SparseGradLinear(10, 20)
MyLayer.weight = nn.Parameter(a, requires_grad = True)
MyLayer.bias = nn.Parameter(c, requires_grad = True)

In [26]:
gt = torch.nn.Parameter(torch.randn((1, 10)))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(MyLayer.parameters())

In [27]:
for i in range(30):
    optimizer.zero_grad()
    output = MyLayer(x)
    loss = criterion(output, gt)
  
    loss.backward()
    MyLayer.grads.append(MyLayer.weight.grad.unsqueeze(0)) ## to backward!!! передать по ссылке self.grads??

    optimizer.step()

In [1]:
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer)

axlen = 512
query_maxlen = 512
doc_maxlen = 512
dim = 128
similarity = 'cosine'

config = AutoConfig.from_pretrained(
        'bert-base-uncased',
        num_labels=2,
        use_auth_token=None)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

bert = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',
        config=config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [5]:
bert.bert.encoder.layer

ModuleList(
  (0): BertLayer(
    (attention): BertAttention(
      (self): BertSelfAttention(
        (query): Linear(in_features=768, out_features=768, bias=True)
        (key): Linear(in_features=768, out_features=768, bias=True)
        (value): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (output): BertSelfOutput(
        (dense): Linear(in_features=768, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (intermediate): BertIntermediate(
      (dense): Linear(in_features=768, out_features=3072, bias=True)
      (intermediate_act_fn): GELUActivation()
    )
    (output): BertOutput(
      (dense): Linear(in_features=3072, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (1): BertLayer

In [41]:
def replace_bert_layers(model):
    if hasattr(model, "bert") and hasattr(model.bert, "encoder"):
        encoder = model.bert.encoder
    elif hasattr(model, "encoder"):
        encoder = model.encoder
    else:
        raise ValueError("Expected model to have attribute 'encoder' or 'bert.encoder'.")

    for i, layer in enumerate(encoder.layer):
        token_dim, hidden_dim = layer.intermediate.dense.weight.shape
        #print ("dense")
        #print ("old shape", token_dim, hidden_dim)
        
        new_layer = SparseGradLinear(token_dim, hidden_dim)

        new_layer.from_linear(layer.intermediate.dense)

        layer.intermediate.dense = new_layer
          
        #print ("new shape", layer.intermediate.dense.weight.shape)
        
        token_dim, hidden_dim = layer.output.dense.weight.shape
        #print ("output")
        #print ("old shape", token_dim, hidden_dim)
        
        new_layer = SparseGradLinear(token_dim, hidden_dim)

        new_layer.from_linear(layer.output.dense)

        layer.intermediate.dense = new_layer
          
        #print ("new shape", layer.output.dense.weight.shape)
        #print ("\n\n")

    return model

In [42]:
replace_bert_layers(bert)

dense
old shape 3072 768
new shape torch.Size([3072, 768])
output
old shape 768 3072
new shape torch.Size([768, 3072])



dense
old shape 3072 768
new shape torch.Size([3072, 768])
output
old shape 768 3072
new shape torch.Size([768, 3072])



dense
old shape 3072 768
new shape torch.Size([3072, 768])
output
old shape 768 3072
new shape torch.Size([768, 3072])



dense
old shape 3072 768
new shape torch.Size([3072, 768])
output
old shape 768 3072
new shape torch.Size([768, 3072])



dense
old shape 3072 768
new shape torch.Size([3072, 768])
output
old shape 768 3072
new shape torch.Size([768, 3072])



dense
old shape 3072 768
new shape torch.Size([3072, 768])
output
old shape 768 3072
new shape torch.Size([768, 3072])



dense
old shape 3072 768
new shape torch.Size([3072, 768])
output
old shape 768 3072
new shape torch.Size([768, 3072])



dense
old shape 3072 768
new shape torch.Size([3072, 768])
output
old shape 768 3072
new shape torch.Size([768, 3072])



dense
old shape 3072 768

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [18]:
for i in range(30):
    optimizer.zero_grad()
    output = SparseGradLinear(x)
    loss = criterion(output, gt)
  
    loss.backward()
    SparseGradLinear.grads.append(SparseGradLinear.weight.grad.unsqueeze(0)) ## to backward!!! передать по ссылке self.grads??

    optimizer.step()

TypeError: __init__() missing 1 required positional argument: 'out_features'

In [27]:
optimizer.zero_grad()
output = MyLayer(x)
loss = criterion(output, gt)

loss.backward()

optimizer.step()

RuntimeError: expected scalar type float but found long int

In [13]:
MyLayer.weight

Parameter containing:
tensor([[-6.3397e-01,  1.4621e+00, -6.2823e-01,  6.3819e-01,  2.3741e+00,
         -1.0120e+00,  2.4445e-03,  1.0348e-01, -1.6454e+00, -5.7057e-01],
        [-1.0153e-01,  4.2967e-02, -1.2769e+00, -1.0584e-01, -3.1527e-02,
          8.5619e-01, -1.6373e-01, -1.1814e+00, -5.6372e-01,  1.5032e-02],
        [ 1.0561e+00, -9.3535e-02, -1.7509e+00,  9.4875e-01,  9.9599e-01,
         -2.0284e+00, -3.6505e-01, -1.1277e+00, -2.0053e+00, -2.7345e-01],
        [-2.2939e+00,  3.9443e-01, -7.3975e-01, -2.1963e-01, -6.9729e-01,
          1.0510e-01, -1.3321e+00,  4.1697e-01,  4.0370e-01,  7.5893e-01],
        [ 9.5387e-02, -1.7673e+00,  1.5727e+00, -1.2527e+00,  6.7039e-01,
          6.6192e-01,  2.6757e-01,  8.7703e-01, -2.2693e-01, -4.6524e-01],
        [ 9.9008e-02, -3.4288e-01, -2.5089e-01, -1.4115e+00,  1.4680e+00,
          3.3700e-01,  5.9032e-02, -5.0326e-01,  1.3245e+00, -5.8610e-01],
        [-7.7166e-02,  1.4623e+00, -2.2558e+00,  3.1157e-01,  5.0954e-01,
         -